In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cafa-5-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-5-protein-function-prediction/IA.txt
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo


In [35]:
!pip install seqio

In [38]:
!pip install obonet

In [39]:
from Bio import SeqIO
import networkx # obonet
import obonet

In [65]:

train_seq = SeqIO.parse("/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta", "fasta")
print(type(train_seq))
print(train_seq)

test_seq = SeqIO.parse("/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta", "fasta")
print(type(test_seq))
print(test_seq)

<class 'Bio.SeqIO.FastaIO.FastaIterator'>
<class 'Bio.SeqIO.FastaIO.FastaIterator'>


In [48]:
i = 0

for s in train_seq:
    if i == 5: break
    
    print(type(s))  # <class 'Bio.SeqRecord.SeqRecord'>
    print(s)
    i += 1

<class 'Bio.SeqRecord.SeqRecord'>
ID: P77596
Name: P77596
Description: P77596 sp|P77596|YAGF_ECOLI D-xylonate dehydratase YagF OS=Escherichia coli (strain K12) OX=83333 GN=yagF PE=1 SV=1
Number of features: 0
Seq('MTIEKIFTPQDDAFYAVITHAAGPQGALPLTPQMLMESPSGNLFGMTQNAGMGW...LGI')
<class 'Bio.SeqRecord.SeqRecord'>
ID: Q16787
Name: Q16787
Description: Q16787 sp|Q16787|LAMA3_HUMAN Laminin subunit alpha-3 OS=Homo sapiens OX=9606 GN=LAMA3 PE=1 SV=3
Number of features: 0
Seq('MAAAARPRGRALGPVLPPTPLLLLVLRVLPACGATARDPGAAAGLSLHPTYFNL...PDQ')
<class 'Bio.SeqRecord.SeqRecord'>
ID: Q59VP0
Name: Q59VP0
Description: Q59VP0 sp|Q59VP0|BST1_CANAL GPI inositol-deacylase OS=Candida albicans (strain SC5314 / ATCC MYA-2876) OX=237561 GN=BST1 PE=3 SV=2
Number of features: 0
Seq('MLSKRLFPYSNLFPRSRKYKFIVYFIICLTIIISALGVYLYSIPIVSPNQPQCD...PTH')
<class 'Bio.SeqRecord.SeqRecord'>
ID: P13508
Name: P13508
Description: P13508 sp|P13508|GLP1_CAEEL Protein glp-1 OS=Caenorhabditis elegans OX=6239 GN=glp-1 PE=1 SV=1
Number o

In [66]:
i = 0

for s in test_seq:
    if i == 5: break
    
    print(type(s))  # <class 'Bio.SeqRecord.SeqRecord'>
    print(s)
    i += 1

<class 'Bio.SeqRecord.SeqRecord'>
ID: Q9CQV8
Name: Q9CQV8
Description: Q9CQV8	10090
Number of features: 0
Seq('MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLLSVAYKNVV...GEN')
<class 'Bio.SeqRecord.SeqRecord'>
ID: P62259
Name: P62259
Description: P62259	10090
Number of features: 0
Seq('MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLSVAYKNVIG...ENQ')
<class 'Bio.SeqRecord.SeqRecord'>
ID: P68510
Name: P68510
Description: P68510	10090
Number of features: 0
Seq('MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLSVAYKNVVG...EGN')
<class 'Bio.SeqRecord.SeqRecord'>
ID: P61982
Name: P61982
Description: P61982	10090
Number of features: 0
Seq('MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLSVAYKNVVG...GNN')
<class 'Bio.SeqRecord.SeqRecord'>
ID: O70456
Name: O70456
Description: O70456	10090
Number of features: 0
Seq('MERASLIQKAKLAEQAERYEDMAAFMKSAVEKGEELSCEERNLLSVAYKNVVGG...PQS')


In [41]:
go_basic_graph = obonet.read_obo("/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo")

In [42]:
len(go_basic_graph)

43248

In [47]:
id_to_name = {id_: data.get('name') for id_, data in go_basic_graph.nodes(data=True)}

# for i, name in enumerate(id_to_name):
#     if i == 5 : break
        
#     print(name)

for i, data in enumerate(go_basic_graph.nodes(data=True)):
    if i == 5 : break
        
    print(data)

('GO:0000001', {'name': 'mitochondrion inheritance', 'namespace': 'biological_process', 'def': '"The distribution of mitochondria, including the mitochondrial genome, into daughter cells after mitosis or meiosis, mediated by interactions between mitochondria and the cytoskeleton." [GOC:mcc, PMID:10873824, PMID:11389764]', 'synonym': ['"mitochondrial inheritance" EXACT []'], 'is_a': ['GO:0048308', 'GO:0048311']})
('GO:0000002', {'name': 'mitochondrial genome maintenance', 'namespace': 'biological_process', 'def': '"The maintenance of the structure and integrity of the mitochondrial genome; includes replication and segregation of the mitochondrial chromosome." [GOC:ai, GOC:vw]', 'is_a': ['GO:0007005']})
('GO:0000003', {'name': 'reproduction', 'namespace': 'biological_process', 'alt_id': ['GO:0019952', 'GO:0050876'], 'def': '"The production of new individuals that contain some portion of genetic material inherited from one or more parent organisms." [GOC:go_curators, GOC:isa_complete, GOC

In [58]:
!ls "/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/"

testsuperset-taxon-list.tsv  testsuperset.fasta


In [60]:
train_taxon = pd.read_csv("/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv", delimiter='\t')
train_terms = pd.read_csv("/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv", delimiter='\t')
sample_submission = pd.read_csv("/kaggle/input/cafa-5-protein-function-prediction/sample_submission.tsv", delimiter='\t')

test_superset_taxon = pd.read_csv("/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv", delimiter='\t', encoding="CP949")

In [ ]:
print(test_superset_taxon.head(5))
print(test_superset_taxon.shape)

In [62]:
print(train_taxon.head(5))
print(train_taxon.shape)

print(train_terms.head(5))
print(train_terms.shape)

print(sample_submission.head(5))
print(sample_submission.shape)



  EntryID  taxonomyID
0  Q8IXT2        9606
1  Q04418      559292
2  A8DYA3        7227
3  Q9UUI3      284812
4  Q57ZS4      185431
(142246, 2)
      EntryID        term aspect
0  A0A009IHW8  GO:0008152    BPO
1  A0A009IHW8  GO:0034655    BPO
2  A0A009IHW8  GO:0072523    BPO
3  A0A009IHW8  GO:0044270    BPO
4  A0A009IHW8  GO:0006753    BPO
(5363863, 3)
   A0A0A0MRZ7  GO:0000001  0.123
0  A0A0A0MRZ7  GO:0000002  0.123
1  A0A0A0MRZ8  GO:0000001  0.123
2  A0A0A0MRZ8  GO:0000002  0.123
3  A0A0A0MRZ9  GO:0000001  0.123
4  A0A0A0MRZ9  GO:0000002  0.123
(283729, 3)
      ID                           Species
0   9606           homo sapiens[All Names]
1  10090           mus musculus[All Names]
2  10116                 Rattus norvegicus
3   3702   Arabidopsis thaliana[All Names]
4  83333  Escherichia coli K-12[all names]
(90, 2)
